numpy and scipy and how to get the saturation point from a cubic EoS
==
This notebook intends to present van der Waals's Equation of State([wikipedia](https://en.wikipedia.org/wiki/Van_der_Waals_equation),[nobel prize lecture](http://www.nobelprize.org/nobel_prizes/physics/laureates/1910/waals-lecture.pdf)), and to derive get density for given values of Pressure and Temperature.

> **Here you will meet the following topics:**
>- Thermodynamics: `equation of state`, `density`,  `saturation point`
>- Python: `packages`, `numpy`, `scipy`

Here we will use more resources of the packages numpy and we will also use the package scipy

In [1]:
import numpy as numpy

the following slightly different syntax works similarly,
here, we impor only a subset of the resources of scipy

In [2]:
from scipy import optimize as optimize
#keyword "from"
     #name of the parent module
           #keyword "import"
                  #name of the desired subpackage
                           #keyword "as"
                              #alias name

to use some function of this module we have to write 'alias name'.'function name'
as an example we will see below:
numpy.roots, numpy.log, numpy.where, optimize.bissect

Supose you have to solve the following problem in a thermodynamics course:
>"Calculate saturation pressure for pure hexane at temperature of 220 K according to van der Waals equation of state"

We will show how here to implement a function to calculate density of a pure fluid at given T and P.

Consider the *van der Waals* equation of state (Reid et al., 1987, see Eq 3-6.1 and Table 3-5, line 1)

$P=\frac{RT}{\bar{V}-b}-\frac{a}{{\bar{V}}^{2}}$

$a=\frac{27}{64}\frac{{{Tc}^{2}}{R^{2}}}{Pc}$


$b=\frac{{R}{Tc}}{{8}{Pc}}$

where $R$ is the gas constant 

$R = 8.3144598 \mathrm{{J} {mol^{−1}} {K^{−1}}}$

and critical properties for hexane are 

$Tc = 507.5 \mathrm{K}$


$Pc = 30.1 \times {10^6} \mathrm{Pa}$

So lets code this information:

First the gas constant

In [3]:
#gas constant
R = 8.3144598 #J.mol^−1.K^−1

now the physical properties of the substance

In [4]:
#pure component critical point of hexane
# Tc (K)
Tc = 507.5 #K

# Pc (bar)
Pc = 30.1*(10**6) #Pa

Then the EoS parameters

In [5]:
a = 27/64*((Tc**2)*(R**2)/Pc)
print(a)

0.2495499363743344


In [6]:
b = (R*Tc)/(8*Pc)
print(b)

1.752320742732558e-05


We define the temperature at which we want to perform calculations

In [7]:
#calculate density at what temperature and pressure?
T = 220. #K
P = 1*(10**5) #Pa

Now instead of using the explixit in P(TV) representation of the EoS we use the implict representation, that is a 3rd order polynomial in V

${P}{{\bar{V}}^3}−({P}{b}+{R}{T}){{\bar{V}}^2}+{a}{\bar{V}}−{a}{b}={0}$

This third grade polynomial provides the values of the volume at a given temperature and pressure.

In [8]:
c3 = P #coefficient for v^3
c2 = -(P*b+R*T) #coefficient v^2
c1 = a  #coefficient v^1
c0 = -a*b  #coefficient v^0

density at given P,T can be obtained by solving this polynomial for V

In [9]:
v=numpy.roots([c3,c2,c1,c0]) #numpy .roots finds the roots of a polynomial
#coefficients must be provided in a list form, constructed using the square brackets
#result is the list of real and imaginary roots found.
print(v)

[  1.81721416e-02   1.16545544e-04   2.06475858e-05]


then calc phi to know whic root has lower gibbs energy and is therefor stable

In [10]:
Residual_G = (P*v)-(R*T)-(a/v) - T*(R*numpy.log(P*(v-b)/(R*T))) #pdf do Charlles
print(Residual_G)

[  -11.92852652  5587.49740938  1954.82586937]


what follows is similar to topliss [XXX] paper:

Calculate Saturation prewssure
for the given T
first we isolate the range of pressure where fluid phase equilibria is feasible:
that is the range of pressure where there are 3 real roots of volume
that is between P_max and P_min
which are pressure values where isotherms P(T,V) have a local maximu and minimum with V belonging to [b,oo)

 


given

$T$,

find all possible

$\bar{V}^*$,

so that

$\left(\frac{\partial P}{  \partial \bar{V}}\right)_{T}=0$

in

$\bar{V}^* \in (b,\infty)$.

Then, find the corresponding

$P^* = P(T,\bar{V}^*)$.

analytical development of the partial derivative yields

$\left(\frac{\partial P}{  \partial \bar{V}}\right)_{T}=-\frac{RT}{(\bar{V}-b)^2} + \frac{2a}{\bar{V}^3}$

therefore in the stationary points

$-\frac{RT}{(\bar{V}-b)^2} + \frac{2a}{\bar{V}^3}=0$

which represents another 3rd order in V polynomial (not the same from the problem of finding V given P and T)

$-{RT}{\bar{V}^3} + {2a}{(\bar{V}-b)^2}=0$

$-{RT}{\bar{V}^3} + {2a}{(\bar{V})^2}+{2a}{(b)^2}+{2a}{(-2\bar{V}b)}=0$

In [14]:
#calculate each coeffcient of the polynomial-in-V representation of dP/dV=0
rho = R*T
mu = 2*(-a)
nu = - b*(a*(-4))
omega=- 2*(b**2)*(a*(1))



In [15]:
#use the method "roots" from package "numpy" to calculate the roots of this polynomial and assing the solutions to the variable plateaus as an array
plateaus = numpy.roots([rho,mu,nu,omega])
print("plateaus:", plateaus)


#filter out values of plateu correspondign to non-physical volume (less than b)
plateaus = plateaus[ numpy.where( plateaus > b ) ]

print("plateaus:", plateaus)
Vli = min(plateaus)
Vvi = max(plateaus)

#find the corrsponding values of pressure for each filtered volume
Psup = R*T/(Vvi-b)-a/(Vvi*Vvi)
Psupi=Psup*.999

Pinf = R*T/(Vli-b)-a/(Vli*Vli)

#note that the algorithm cannot work with pressure less than or equal to zero, therefore fix minimium pressure to slightly above zero if is resulted negative

if Pinf > 0:
    Pinfi = Pinf*1.001
else:
    Pinfi = 0.000000001

print("Vli:",Vli)
print("Vvi:",Vvi)
print("Psupi:",Psupi)
print("Pinfi:",Pinfi)     



plateaus: [  2.33425728e-04   2.51661243e-05   1.42624090e-05]
plateaus: [  2.33425728e-04   2.51661243e-05]
Vli: 2.51661242908e-05
Vvi: 0.000233425727852
Psupi: 3888420.00342
Pinfi: 1e-09


calc hres
that comes from the following integration
$$Hres = RT(-T(\int{de 0 a P, dzdt a p dp sobre p}))$$
we show how to change into integration in dV
$$[n.a.]$$
that yields the following expresion specifically for vdWnP EoS
$$Hres=PV-RT+(TdadT-a)/V$$

calc Gres ("J" omitted)
that comes from the following integration
$$Gres = RT(Gres@P=0 + \int {de 0 a P, (V-V^(GI)/RT) dP,})$$
we show how to change into integration in dV
$$[n.a.]$$
and that yields the following expresion specifically for vdWnP EoS
$$Sres=PV-RT+(TdadT-a)/V$$

calc Sres from subtraction S=(H-G)/T

REfs:
Smith van Ness Abott, 7th ed, ch 6
O connell e Haile, ed 1005 ch 4
TEster modell 1997 ch 9

calcpsat
to calc psat we will need to find P that makes  delta_Residual_G nequal zero
as we cannot explicitate P as a function of delta_Residual_G
for that purpose we define of a function delta_Residual_G of P
and we will solve it numerically, i.e. tsting various values of P
t automate taht task we impkement a function **`def Delta_Residual_G(P):`**

In [13]:



#define equilibrium criteria: Residual_G_Liquid is equal to Residual_G_Vapor :. Delta_Residual_G = zero :. Delta_Residual_G < Tolerance
def Delta_Residual_G(P):
    alfa = (-1)*b-R*T/P
    beta = + a/P
    gamma = -b*a/P
    V = numpy.roots([1,alfa,beta,gamma])
    Vl = min(V)
    Vv = max(V)
    #print(P, V) #if you uncomment this line you will see every iteration in the output section of this cell
    return (P*(Vl-Vv) + (-a)*(1/(Vl)-1/(Vv))+T*(R*numpy.log((Vv-b)/(Vl-b))))


Saturation pressure = [ 297561.1280524415 ] bar


now we must find the solution P
we ask package optmize to use method bissect to solve the equilibrium criteria just defined


In [ ]:

psat = optimize.bisect(Delta_Residual_G, Pinfi, Psupi, xtol=1e-12, rtol=4.4408920985006262e-16, maxiter=100, full_output=True, disp=True)
print("Saturation pressure = [",str(psat[0]),"] bar")



# External references
scipy, optimize [documentation](http://docs.scipy.org/doc/scipy/reference/optimize.html)

# Credits
* Initially developed in python 2.7 by Guilherme Carneiro Queiroz da Silva
* Adapted to ipynb with python 3.5 by Iuri Soter Viana Segtovich